# Scan sessions
Sessions are scanned to generate maps between image files and their EXIF dates. Those maps can later be used to get the closest Lapse images for a given Motion image for frame differencing or other comparison methods.

In [28]:
%load_ext autoreload
%autoreload 2

from tqdm.notebook import tqdm
import os
from datetime import datetime

from py.Dataset import Dataset
from py.DatasetStatistics import DatasetStatistics
from py.Session import Session
from py.FileUtils import list_folders, list_jpegs_recursive

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [63]:
DIR = '/home/AMMOD_data/camera_traps/BayerWald/Vielkadaver-Projekt/'

ds = Dataset(DIR)

Found 32 sessions


In [64]:
print(ds.get_sessions())

['Beaver_01', 'Marten_01', 'Raccoon_01', 'Reddeer_01', 'Roedeer_01', 'Wildboar_01', 'Badger_02', 'Beaver_02', 'Fox_02', 'Marten_02', 'Raccoon_02', 'Rat_02', 'Reddeer_02', 'Roedeer_02', 'Badger_03', 'Fox_03', 'Raccoon_03', 'Reddeer_03', 'Wildboar_03', 'Badger_04', 'Rat_04', 'Reddeer_04', 'Wildboar_04', 'Badger_05', 'Beaver_05', 'Ermine_05', 'Fox_05', 'Marten_05', 'Raccoon_05', 'Reddeer_05', 'Roedeer_05', 'Wildboar_05']


Sessions can be created from the dataset. On instancing, the session looks for previous session scans in the session_scans folder and loads them automatically if found.

In [67]:
session = ds.create_session("Marten_01")

Session 'Marten_01' at folder: /home/AMMOD_data/camera_traps/BayerWald/Vielkadaver-Projekt/VIELAAS_Spring_Session01-VIELAAS_Marten_01
Loaded scans.


If the session has not been scanned:

In [52]:
session.scan()

Scanning motion dates...


100%|██████████| 845/845 [00:16<00:00, 51.74it/s]


Scanning lapse dates...


100%|██████████| 626/626 [00:13<00:00, 47.86it/s]

Creating lapse map...
Saving...
Saved session_scans/Rat_02/lapse_dates.pickle
Saved session_scans/Rat_02/motion_dates.pickle
Saved session_scans/Rat_02/lapse_map.pickle


## Duplicates and inconsistent duplicates
The following functions checks for consistent and inconsistent (deviant) duplicates in the session. Duplicates are two or more images with the same EXIF date. Duplicates are inconsistent when they show different scenes (estimated by comparing the JPEG file size) despite being taken at the same time (and, supposedly, the same place).

In [53]:
total, total_duplicates, _, deviants = session.check_lapse_duplicates()

100%|██████████| 578/578 [00:00<00:00, 40275.58it/s]

* 578 lapse dates
* 48 duplicates
* 0 multiples (more than two files per date)
* 0 deviant duplicates: []


Open and compare the duplicate images for the same date.

In [68]:
session.open_images_for_date(datetime(2021, 6, 22, 9, 0))

#1 /home/AMMOD_data/camera_traps/BayerWald/Vielkadaver-Projekt/VIELAAS_Spring_Session01-VIELAAS_Marten_01/Lapse/2021_06_22_09h_5050.jpg
#2 /home/AMMOD_data/camera_traps/BayerWald/Vielkadaver-Projekt/VIELAAS_Spring_Session01-VIELAAS_Marten_01/Lapse/2021_06_22_09h_5348.jpg


### Stats

| Session | Total lapse dates | Duplicates | Multiples | Inconsistent Duplicates |
| --- | --- | --- | --- | --- |
| Beaver_01 | 1734 | 0 | 0 | 0 |
| Marten_01 | 1733 | 729 | 0 | 108 |
| Raccoon_01 | 850 | 0 | 0 | 0 |
| Reddeer_01 | 1628 | 0 | 0 | 0 |
| Roedeer_01 | 1353 | 27 | 0 | 0 |
| Wildboar_01 | 1732 | 0 | 0 | 0 |
| Badger_02 | 1728 | 0 | 0 | 0 |
| Beaver_02 | 1727 | 0 | 0 | 0 |
| Fox_02 | 957 | 0 | 0 | 0 |
| Marten_02 | 1726 | 0 | 0 | 0 |
| Raccoon_02 | 1639 | 0 | 0 | 0 |
| Rat_02 | 578 | 48 | 0 | 0 |
| Reddeer_02 | 1635 | 0 | 0 | 0 |
| Roedeer_02 | 888 | 0 | 0 | 0 |
| Badger_03 | 46 | 0 | 0 | 0 |
| Fox_03 | 34 | 1 | 1 | 1 |
| Raccoon_03 | 35 | 0 | 0 | 0 |
| Reddeer_03 | 1027 | 0 | 0 | 0 |
| Wildboar_03 | 46 | 0 | 0 | 0 |
| Badger_04 | 56 | 0 | 0 | 0 |
| Rat_04 | 39 | 0 | 0 | 0 |
| Reddeer_04 | 46 | 0 | 0 | 0 |
| Wildboar_04 | 39 | 0 | 0 | 0 |
| Badger_05 | 1105 | 69 | 0 | 1 |
| Beaver_05 | 1105 | 216 | 0 | 0 |
| Ermine_05 | 867 | 0 | 0 | 0 |
| Fox_05 | 1083 | 0 | 0 | 0 |
| Marten_05 | 888 | 2 | 0 | 2 |
| Raccoon_05 | 1088 | 1 | 0 | 1 |
| Reddeer_05 | 1078 | 0 | 0 | 0 |
| Roedeer_05 | 1090 | 0 | 0 | 0 |
| Wildboar_05 | 902 | 1 | 0 | 1 |


In [60]:
# code for generating the markup for the table above

out_file = None

for sess_name in ds.get_sessions():
    session = ds.create_session(sess_name)
    if not session.scanned:
        session.scan()
    total, total_duplicates, total_multiples, deviants = session.check_lapse_duplicates()
    out = f"| {sess_name} | {total} | {total_duplicates} | {total_multiples} | {len(deviants)} |"
    print(out)
    if out_file is not None:
        with open(out_file, "a") as f:
            f.write(f"{out}\n")

Session 'Beaver_01' at folder: /home/AMMOD_data/camera_traps/BayerWald/Vielkadaver-Projekt/VIELAAS_Spring_Session01-VIELAAS_Beaver_01
Loaded scans.


100%|██████████| 1734/1734 [00:00<00:00, 792430.06it/s]


* 1734 lapse dates
* 0 duplicates
* 0 multiples (more than two files per date)
* 0 deviant duplicates: []
| Beaver_01 | 1734 | 0 | 0 | 0 |
Session 'Marten_01' at folder: /home/AMMOD_data/camera_traps/BayerWald/Vielkadaver-Projekt/VIELAAS_Spring_Session01-VIELAAS_Marten_01
Loaded scans.


100%|██████████| 1733/1733 [00:00<00:00, 3852.62it/s]


* 1733 lapse dates
* 729 duplicates
* 0 multiples (more than two files per date)
* 108 deviant duplicates: [datetime.datetime(2021, 6, 17, 23, 0), datetime.datetime(2021, 6, 18, 0, 0), datetime.datetime(2021, 6, 18, 1, 0), datetime.datetime(2021, 6, 18, 2, 0), datetime.datetime(2021, 6, 18, 3, 0), datetime.datetime(2021, 6, 18, 4, 0), datetime.datetime(2021, 6, 18, 5, 0), datetime.datetime(2021, 6, 18, 6, 0), datetime.datetime(2021, 6, 18, 7, 0), datetime.datetime(2021, 6, 18, 8, 0), datetime.datetime(2021, 6, 18, 9, 0), datetime.datetime(2021, 6, 18, 10, 0), datetime.datetime(2021, 6, 18, 11, 0), datetime.datetime(2021, 6, 18, 12, 0), datetime.datetime(2021, 6, 18, 13, 0), datetime.datetime(2021, 6, 18, 14, 0), datetime.datetime(2021, 6, 18, 15, 0), datetime.datetime(2021, 6, 18, 16, 0), datetime.datetime(2021, 6, 18, 17, 0), datetime.datetime(2021, 6, 18, 18, 0), datetime.datetime(2021, 6, 18, 19, 0), datetime.datetime(2021, 6, 18, 20, 0), datetime.datetime(2021, 6, 18, 21, 0), datet

...

Scanning lapse dates...


100%|██████████| 903/903 [00:21<00:00, 42.06it/s]


Creating lapse map...
Saving...
Saved session_scans/Wildboar_05/lapse_dates.pickle
Saved session_scans/Wildboar_05/motion_dates.pickle
Saved session_scans/Wildboar_05/lapse_map.pickle


100%|██████████| 902/902 [00:00<00:00, 442642.12it/s]

* 902 lapse dates
* 1 duplicates
* 0 multiples (more than two files per date)
* 1 deviant duplicates: [datetime.datetime(2021, 6, 13, 15, 0)]
| Wildboar_05 | 902 | 1 | 0 | 1 |
